In [2]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats





from pymongo import MongoClient







from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [10]:

%time df_citation = pickle.load(open('../data/ref_dataframe_min.pkl', 'rb'))   # citation_dataframe.pkl
print ("done loading pickles", df_citation.shape)



df_citation = df_citation[df_citation['reference_UT'] != '-1']   # i dont know why, but there are 7 occurrences with value -1


print ("done removing ref_UT = -1 instances:", df_citation.shape)


df_citation.head()


# lista_ref_UTs = list(df_citation.reference_UT.unique() )

CPU times: user 19.5 s, sys: 6.08 s, total: 25.6 s
Wall time: 1min 59s
done loading pickles (10848620, 21)
done removing ref_UT = -1 instances: (8907763, 21)


,occurence,paper_UT,paper_char_pos,paper_word_pos,para_char_pos,para_char_total,para_index,para_word_pos,para_word_total,reference_UT,...,sect_char_pos,sect_char_total,sect_index,sect_word_pos,sect_word_total,section_title,section_title_alt,regex_sect_index,ref_datetime,plos_datetime
0,1,324515600133,139,21,139,820,0,21,130,A1995QY75100004,...,139,4029,0,21,658,Introduction,introduction,0,1995-05-15 00:00:00,2013-09-04 00:00:00
1,2,324515600133,16829,2992,494,1208,24,76,188,A1995QY75100004,...,494,5398,3,76,864,Discussion,discussion,3,1995-05-15 00:00:00,2013-09-04 00:00:00
2,3,324515600133,17257,3057,922,1208,24,141,188,A1995QY75100004,...,922,5398,3,141,864,Discussion,discussion,3,1995-05-15 00:00:00,2013-09-04 00:00:00
3,1,324515600133,266,42,266,820,0,42,130,000222499800017,...,266,4029,0,42,658,Introduction,introduction,0,2004-06-15 00:00:00,2013-09-04 00:00:00
4,2,324515600133,444,73,444,820,0,73,130,000222499800017,...,444,4029,0,73,658,Introduction,introduction,0,2004-06-15 00:00:00,2013-09-04 00:00:00


In [57]:
##### i simplify the file, dropping all columns i dont need, dropping duplicates and reindexing df

len(df_citation.reference_UT.unique() ) # 2,607,457

len(df_citation.paper_UT.unique() ) # 158,813


df_ref_one_per_sect = df_citation.drop_duplicates(subset=['reference_UT', 'paper_UT','sect_index'])
df_ref_one_per_sect = df_ref_one_per_sect[['reference_UT', 'paper_UT','sect_index']] 


df_ref_one_per_sect = df_ref_one_per_sect.rename(columns={'sect_index': 'section', 'paper_UT': 'WoS_ID_plos', 'reference_UT': 'WoS_ID_reference'})

df_ref_one_per_sect.shape    # 6,930,676


df_ref_one_per_sect.reset_index(drop=True, inplace=True)
df_ref_one_per_sect



,WoS_ID_reference,WoS_ID_plos,section
0,A1995QY75100004,324515600133,0
1,A1995QY75100004,324515600133,3
2,000222499800017,324515600133,0
3,000222499800017,324515600133,3
4,000263911400006,324515600133,0
5,000289279600018,324515600133,0
6,000289279600018,324515600133,3
7,000287717700023,324515600133,0
8,000257589600013,324515600133,0
9,000257589600013,324515600133,3


In [58]:
### i fix the issue of the missing 000 in the paper_UTs


def get_new_paper_UT(row):
    
    new_UT= '000'+str(row.WoS_ID_plos)
   
    return new_UT

###################


%time df_ref_one_per_sect['new_paper_UT'] = df_ref_one_per_sect.apply (lambda row: get_new_paper_UT(row),axis=1)

df_ref_one_per_sect.drop(['WoS_ID_plos'], axis=1, inplace=True)
df_ref_one_per_sect.rename(columns={'new_paper_UT': 'WoS_ID_plos'}, inplace=True)

CPU times: user 1min 35s, sys: 249 ms, total: 1min 35s
Wall time: 1min 37s


In [59]:
df_ref_one_per_sect.dtypes

WoS_ID_reference    object
section              int64
WoS_ID_plos         object
dtype: object

In [60]:
df_ref_one_per_sect

,WoS_ID_reference,section,WoS_ID_plos
0,A1995QY75100004,0,000324515600133
1,A1995QY75100004,3,000324515600133
2,000222499800017,0,000324515600133
3,000222499800017,3,000324515600133
4,000263911400006,0,000324515600133
5,000289279600018,0,000324515600133
6,000289279600018,3,000324515600133
7,000287717700023,0,000324515600133
8,000257589600013,0,000324515600133
9,000257589600013,3,000324515600133


In [61]:

#### i print out the pandas dataframe
with open('../data/df_plos_WoS_id_refs_sects_for_public.pkl', 'wb') as handle:
    pickle.dump(df_ref_one_per_sect, handle, protocol = 2)
print ("written:",'../data/df_plos_WoS_id_refs_sects_for_public.pkl',len(df_ref_one_per_sect))   



#### create json files with basic info from the original dataframe, for sharing at publication
df_ref_one_per_sect.to_json(path_or_buf = '../data/df_plos_WoS_id_refs_sects_for_public.json', orient= 'records')  # check out documentation for df.to_json()  # check out documentation for 'orient = split vs records'

print ('written:  ../data/df_plos_WoS_id_refs_sects_for_public.json')



df_ref_one_per_sect.to_csv(path_or_buf='../data/df_plos_WoS_id_refs_sects_for_public.csv', sep=';', na_rep='')
print ('written:  ../data/df_plos_WoS_id_refs_sects_for_public.csv')

written: ../data/df_plos_WoS_id_refs_sects_for_public.pkl 6930676
written:  ../data/df_plos_WoS_id_refs_sects_for_public.json
written:  ../data/df_plos_WoS_id_refs_sects_for_public.csv


In [1]:
testing_df = pd.read_json(path_or_buf='../data/df_plos_WoS_id_refs_sects_for_public.json', orient='records',dtype='str')
testing_df

NameError: name 'pd' is not defined

In [63]:

%time testing_df2 = pickle.load(open('../data/df_plos_WoS_id_refs_sects_for_public.pkl', 'rb'))   # citation_dataframe.pkl
print ("done loading pickles", testing_df2.shape)
testing_df2

CPU times: user 1.56 s, sys: 161 ms, total: 1.73 s
Wall time: 3.13 s
done loading pickles (6930676, 3)


,WoS_ID_reference,section,WoS_ID_plos
0,A1995QY75100004,0,000324515600133
1,A1995QY75100004,3,000324515600133
2,000222499800017,0,000324515600133
3,000222499800017,3,000324515600133
4,000263911400006,0,000324515600133
5,000289279600018,0,000324515600133
6,000289279600018,3,000324515600133
7,000287717700023,0,000324515600133
8,000257589600013,0,000324515600133
9,000257589600013,3,000324515600133


In [43]:
testing_df2.dtypes

WoS_ID_reference    object
section              int64
WoS_ID_plos         object
dtype: object

In [66]:
df_ref_one_per_sect

,WoS_ID_reference,section,WoS_ID_plos
0,A1995QY75100004,0,000324515600133
1,A1995QY75100004,3,000324515600133
2,000222499800017,0,000324515600133
3,000222499800017,3,000324515600133
4,000263911400006,0,000324515600133
5,000289279600018,0,000324515600133
6,000289279600018,3,000324515600133
7,000287717700023,0,000324515600133
8,000257589600013,0,000324515600133
9,000257589600013,3,000324515600133


In [3]:
testing_df3 = pd.read_csv('../data/df_plos_WoS_id_refs_sects_for_public.csv',sep=';',dtype='str', index_col='Unnamed: 0')
print ("done loading pickles", testing_df3.shape)
testing_df3



done loading pickles (6930676, 3)


,WoS_ID_reference,section,WoS_ID_plos
0,A1995QY75100004,0,000324515600133
1,A1995QY75100004,3,000324515600133
2,000222499800017,0,000324515600133
3,000222499800017,3,000324515600133
4,000263911400006,0,000324515600133
5,000289279600018,0,000324515600133
6,000289279600018,3,000324515600133
7,000287717700023,0,000324515600133
8,000257589600013,0,000324515600133
9,000257589600013,3,000324515600133


In [46]:
testing_df3.dtypes

Unnamed: 0          object
WoS_ID_reference    object
section             object
WoS_ID_plos         object
dtype: object